In [ ]:
import lstmcpipe
print(lstmcpipe.__version__)

from lstmcpipe.plots.pointings import plot_pointings

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.io.ascii import read
from pathlib import Path

In [ ]:
plt.style.use('seaborn-paper')
plt.style.use('seaborn-colorblind')

In [ ]:
def groupby_alt(pointings):
    pp = pointings.to_pandas()
    return pp.groupby('alt').groups

In [ ]:
pointings_test = read('pointings_test.ecsv')

In [ ]:
pointings_test

In [ ]:
pointings_train = read('pointings_train.ecsv', format='ecsv')
pointings_train = np.transpose([pointings_train['az'].to(u.rad), pointings_train['alt'].to(u.rad)])*u.rad
pointings_train

In [ ]:
plt.figure(figsize=(5,5))

ax = plot_pointings(pointings_train,
                    label='Training nodes', color='black', s=8)


grp = groupby_alt(pointings_test)

for ii, (k, index) in enumerate(grp.items()):
    index=list(index)
    ax.scatter(pointings_test[index]['az'].to_value(u.rad), np.pi/2. - pointings_test[index]['alt'].to_value(u.rad), 
               marker='*',
               label=f"Testing nodes zd={90*u.deg-pointings_test[index[0]]['alt']:.2f}",
               s=100
              )

rticks = [10, 20, 30, 40, 50, 60]
ax.set_rticks(np.deg2rad(rticks), [f'{r:d}°' for r in rticks])
ax.text(np.radians(ax.get_rlabel_position()+10),2*ax.get_rmax()/3.,'Zenith',
        rotation=ax.get_rlabel_position(),ha='center',va='center')


ax.vlines(np.deg2rad(175.158), 0, 0.99*ax.get_rmax(),
          ls='dotted', 
          color='grey', lw=1, zorder=0,
          label='magnetic North-South'
         )
ax.vlines(np.deg2rad(175.158)+np.pi, 0, 0.99*ax.get_rmax(), ls='dotted', color='grey', lw=1, zorder=0)


ax.legend(fontsize=8, loc='lower right', bbox_to_anchor=(1.1, 0.16))




plt.tight_layout()
plt.savefig(Path('.', 'pointings_per_alt.png'), dpi=250)
plt.show()